In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../data/invoices.csv')


In [3]:
df.head()

,id,Amount Due,Closed,Currency,Customer,Date (UTC),Due Date (UTC),Ending Balance,Forgiven,Paid,...,Subtotal,Total Discount Amount,Applied Coupons,Tax,Tax Percent,Total,Amount Paid,Status,Exclusive Tax Amount,Inclusive Tax Amount
0,in_1Qx3plAFeGZf0UoOd9f9bW47,"0,00",False,eur,cus_NHFLrvkCpBFCs7,2025-02-27 9:56,NaN,NaN,False,False,...,"0,00","0,00",NaN,"0,00",20.0,"0,00","0,00",draft,"0,00","0,00"
1,in_1Qx3phAFeGZf0UoOy7Khkj9i,"0,00",False,eur,cus_LSl85Ix2Ek965l,2025-02-27 9:56,NaN,NaN,False,False,...,"0,00","0,00",NaN,"0,00",19.0,"0,00","0,00",draft,"0,00","0,00"
2,in_1Qx3XqAFeGZf0UoONAAW2muL,"104,40",False,eur,cus_PQFFOsNVNtuBN9,2025-02-27 9:38,2025-03-02 9:37,NaN,False,False,...,"87,00","0,00",NaN,"17,40",20.0,"104,40","0,00",draft,"17,40","0,00"
3,in_1Qx3XqAFeGZf0UoOl79vERDa,"39,00",False,eur,cus_Q7apBCt36xeGbK,2025-02-27 9:38,NaN,NaN,False,False,...,"39,00","0,00",NaN,NaN,NaN,"39,00","0,00",draft,NaN,NaN
4,in_1Qx3WCAFeGZf0UoOcbJhPyor,"82,80",False,eur,cus_MU4dOalEEat1Ah,2025-02-27 9:36,NaN,NaN,False,False,...,"69,00","0,00",NaN,"13,80",20.0,"82,80","0,00",draft,"13,80","0,00"


In [4]:
df.describe()

,Tax Percent
count,3078.000000
mean,20.725276
std,2.285339
min,7.700000
25%,19.000000
50%,20.000000
75%,21.000000
max,27.000000


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8075 entries, 0 to 8074
Data columns (total 30 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   id                                    8075 non-null   object 
 1   Amount Due                            8075 non-null   object 
 2   Closed                                8075 non-null   bool   
 3   Currency                              8075 non-null   object 
 4   Customer                              8075 non-null   object 
 5   Date (UTC)                            8075 non-null   object 
 6   Due Date (UTC)                        224 non-null    object 
 7   Ending Balance                        8037 non-null   object 
 8   Forgiven                              8075 non-null   bool   
 9   Paid                                  8075 non-null   bool   
 10  Paid At (UTC)                         7765 non-null   object 
 11  Marked Uncollecti

In [6]:
# Convert date columns to datetime
date_columns = [
    'Date (UTC)', 'Due Date (UTC)', 'Finalized At (UTC)', 
    'Marked Uncollectible At (UTC)', 'Voided At (UTC)',
    'Minimum Line Item Period Start (UTC)', 'Maximum Line Item Period End (UTC)',
    'Period Start (UTC)', 'Period End (UTC)', 'Paid At (UTC)'
]

for col in date_columns:
    df[col] = pd.to_datetime(df[col], errors='coerce')

# Convert numeric columns (handling European number format with comma as decimal separator)
numeric_columns = [
    'Amount Due', 'Ending Balance', 'Starting Balance', 'Subtotal',
    'Total Discount Amount', 'Tax', 'Total', 'Amount Paid',
    'Exclusive Tax Amount', 'Inclusive Tax Amount'
]

for col in numeric_columns:
    # Replace comma with dot for decimal point and convert to float
    df[col] = df[col].str.replace(',', '.').astype(float)

# Convert percentage columns
df['Tax Percent'] = df['Tax Percent'].astype(float)

# Convert categorical columns
categorical_columns = ['Currency', 'Status']
for col in categorical_columns:
    df[col] = df[col].astype('category')

# Convert all column names to lowercase
df.columns = df.columns.str.lower()

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8075 entries, 0 to 8074
Data columns (total 30 columns):
 #   Column                                Non-Null Count  Dtype         
---  ------                                --------------  -----         
 0   id                                    8075 non-null   object        
 1   Amount Due                            8075 non-null   float64       
 2   Closed                                8075 non-null   bool          
 3   Currency                              8075 non-null   category      
 4   Customer                              8075 non-null   object        
 5   Date (UTC)                            8075 non-null   datetime64[ns]
 6   Due Date (UTC)                        224 non-null    datetime64[ns]
 7   Ending Balance                        8037 non-null   float64       
 8   Forgiven                              8075 non-null   bool          
 9   Paid                                  8075 non-null   bool          
 10  

In [9]:
df.Status

0       draft
1       draft
2       draft
3       draft
4       draft
        ...  
8070     paid
8071     paid
8072     paid
8073     paid
8074     paid
Name: Status, Length: 8075, dtype: object